# IV. 실습: 예측 결과와 시각화

## 강의 목표
- 모델의 예측 결과를 다양한 형태로 시각화합니다.
- 확률 분포, Confusion Matrix 등을 통해 예측 결과를 쉽게 이해할 수 있습니다.
- 데이터 후처리를 통해 **설명가능성(Interpretability)** 를 확보합니다.
- WordCloud로 악성/정상 트래픽의 주요 단어 분포를 파악합니다.

## 4.1 예측 확률값 시각화

모델이 각 샘플에 대해 예측한 "확률값"은 단순한 라벨보다 훨씬 더 많은 정보를 담고 있습니다.  
이 확률값 분포를 시각화함으로써 모델의 **자신감(confidence)**,  
그리고 **불확실성(uncertainty)** 을 직관적으로 이해할 수 있습니다.

예를 들어:
- 확률이 0.95 이상인 샘플은 모델이 **거의 확신하고 있음**
- 확률이 0.5 근처인 샘플은 **혼란스러워함** → 추가 확인 필요

이를 시각화하면 다음과 같은 통찰을 얻을 수 있습니다:
- 모델이 자주 **혼동하는 구간**
- 악성/정상 중 **특정 클래스에 대한 확신도 차이**


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False  

CLASS_NAMES = [
    'Benign',
    'BEanking_motet_Family',
    'Banking_Zeus_Family',
    'Banking_Other',
    'Infostealer',
    'RAT',
    'Exploit_Kit',
    'Malspam_Phishing',
    'Ransomware',
    'Recon_C2',
    'Other_Generic',
    'Cobalt_Strike'
]



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 예측 결과 파일 로딩 (이전 실습에서 생성된 validation_result_all.csv 사용)
df = pd.read_csv("validation_result.csv")

# 예측값, 정답값은 숫자 클래스 (int), 확률은 0~1
df["correct"] = df["target"] == df["output"]

# 전체 확률 분포 히스토그램
plt.figure(figsize=(10, 5))
sns.histplot(df["preds"], bins=50, kde=True)
plt.title("전체 예측 확률값 분포")
plt.xlabel("예측 확률값 (confidence)")
plt.ylabel("샘플 수")
plt.grid(True)
plt.show()

# 클래스별 확률 boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(x="target", y="preds", data=df)
# x축 클래스명으로 대체
plt.xticks(ticks=range(len(CLASS_NAMES)), labels=CLASS_NAMES, rotation=45, ha='right')
plt.title("클래스별 예측 확률 boxplot (target 기준)")
plt.xlabel("실제 클래스 (target)")
plt.ylabel("예측 확률값")
plt.grid(True)
plt.show()


## 

예측 확률값 시각화 해석
### 1. 전체 예측 확률값 분포

- 대부분의 예측 확률이 **0.95 ~ 1.0 구간에 몰려 있음**  
  → 모델이 대부분의 샘플에 대해 **매우 확신(confident)** 하고 있음을 의미  
- 0.5~0.9 사이의 구간은 거의 없음  
  → **애매한 판단(uncertainty)** 은 드문 편  
- 확률값이 낮은 일부 구간(0.2~0.4)에 소수의 샘플 존재  
  → 이는 **오탐/미탐 가능성이 있는 영역**으로 해석할 수 있음

---

### 2. 클래스별 예측 확률 boxplot

- 대부분의 클래스에서 예측 확률이 매우 높음 (상자 대부분이 0.95~1.0)  
  → 모델이 **정답 클래스를 잘 구분하고 있음**  
- 하지만 일부 클래스(예: 클래스 6번 Exploit_Kit)에서는 분산이 큼  
  → 예측 확률이 낮은 샘플이 많아 **모델이 덜 확신하거나 혼동하는 클래스**일 수 있음  
- Outlier 점들이 아래로 길게 뻗은 클래스는 **모델이 종종 불확실한 판단을 내리는 경향이 있음**

- 주요 클래스별 특징 요약
  - `Benign`: 대부분 예측 확률이 0.99 이상으로 모델이 매우 확신하는 경우가 많음. (정상 탐지에 자신 있음)
  - `Exploit_Kit`: 분포가 넓고 박스가 큼 → 모델의 예측 불확실성이 존재함.
  - `Malspam_Phishing`, `Banking_*`: 높은 예측 확률과 작은 분산 → 잘 학습된 패턴.
  - `Recon_C2`, `Other_Generic`, `Cobalt_Strike`: 일부 이상값 존재. 특정 샘플에서 확신이 낮은 경우 있음.

---

#### 활용 방안

- **모델 신뢰성 평가**: 어떤 클래스는 모델이 항상 높은 확신을 가지는 반면, 어떤 클래스는 낮은 확신을 보임.
- **임계값 조정 시 기준**: 예측 확률이 낮은 클래스에 대해서는 threshold를 재조정하거나 후처리 로직을 추가할 수 있음.
- **XAI와 연계**: 왜 특정 클래스의 확률이 낮았는지 원인을 분석하기 위한 기반 자료로 활용 가능

---


### 4.2 Confusion Matrix 실습 (confusion matrix 기반 모델 평가)
- 분류기의 전체 성능을 한눈에 시각화합니다.
- True Positive / False Positive / False Negative / True Negative 비율을 직관적으로 확인합니다.
- 모델이 잘 맞춘 영역과 자주 틀리는 클래스 간 상관관계를 파악할 수 있습니다.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

df = pd.read_csv("validation_result.csv")

y_true = df["target"]
y_pred = df["output"]

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASS_NAMES,
            yticklabels=CLASS_NAMES)
plt.xlabel("예측 클래스 (output)")
plt.ylabel("실제 클래스 (target)")
plt.title("Confusion Matrix (예측 vs 실제)")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


- 대각선에 가까울수록 모델이 정확하게 예측한 경우입니다 (True Positive).
- 비대각선 셀의 값이 클수록, 클래스 간 혼동이 있는 경우입니다.

예시:
- 만약 `Infostealer` (클래스 4)의 행에서 `Benign` (클래스 0) 열의 값이 높다면,
  → Infostealer를 정상으로 잘못 예측한 경우가 많다는 뜻입니다. (False Negative)

- `Benign`의 행에서 다른 악성 클래스 열로 예측된 경우가 많다면,
  → 정상 트래픽을 악성으로 잘못 탐지하는 경우입니다. (False Positive)

이 시각화를 통해:
- 어떤 악성 클래스가 탐지가 잘 되는지 (TP)
- 어떤 클래스가 자주 오탐되는지 (FP/FN)
를 한눈에 파악할 수 있습니다.

## Confusion Matrix 분석 결과 해석

Confusion Matrix는 모델의 예측값과 실제 라벨 간의 관계를 정리한 행렬로, 분류 성능을 직관적으로 파악할 수 있는 도구입니다. 각 셀의 숫자는 `(실제 클래스, 예측 클래스)` 쌍에 해당하는 예측 횟수를 의미합니다.

### 주요 관찰 포인트

- **Benign (정상 트래픽)**  
  - 총 269,332건이 정상으로 정확히 예측됨.
  - 그러나 악성으로 잘못 예측된 경우도 존재 (예: 148건은 Emotet, 13건은 Zeus 등으로 오탐).

- **Banking 계열 (Emotet, Zeus, Other)**  
  - `Banking_Emotet_Family`: 257,297건 정확 예측.
  - `Banking_Zeus_Family`: 일부는 Emotet(1030), Other(10716)과 혼동.
  - `Banking_Other`: 총 116,703건 정확 예측했으나, Zeus(10716), Emotet(10227)와 다소 혼선 있음.

- **Infostealer**  
  - 17,314건을 정확히 예측.
  - 다만 Emotet(1702), Zeus(31) 등으로 일부 잘못 분류됨 → 타 Banking 계열과 혼동 가능성.

- **Exploit_Kit**  
  - 6,473건 정확 예측. 그러나 Benign(778), Emotet(250), Other(134)로 다소 오탐 존재.

- **Malspam_Phishing**  
  - 5,333건 정확 예측.
  - Emotet(678), Zeus(65)와 상당수 혼동 → 비슷한 이메일 기반 특성 가능성.

- **Ransomware**  
  - 1,377건 정확 예측.
  - Benign(193), Emotet(44) 등으로 일부 오탐.

- **Recon_C2 / Other_Generic / Cobalt_Strike**  
  - Recon_C2: 24,104건 정확 예측, 다만 Benign(3,770)으로 많이 오탐됨 → 탐지 기준 약함.
  - Other_Generic: 18,546건 정확 예측. Benign(19), Emotet(1,040)과 일부 혼동.
  - Cobalt_Strike: 11,797건 정확 예측, 하지만 Emotet(1,518), Zeus(72) 등으로 분류된 경우 있음.

---

### 추가 설명

- **대각선이 진할수록** 해당 클래스는 잘 예측되었음을 의미.
- **대각선 바깥의 값**은 오탐(FP), 미탐(FN)에 해당함.
- 특히 `Emotet`, `Zeus`, `Infostealer` 간의 혼동이 빈번하므로 추가적인 전처리/특징 강화 필요.



## 실습: 클래스별 ROC Curve 시각화

이 실습에서는 다중 클래스 분류에서 **각 클래스별 ROC Curve**를 확인합니다.

### 학습 목표
- ROC Curve의 개념을 이해하고 민감도(Recall)와 특이도(Specificity)의 관계 파악
- 모델이 클래스별로 얼마나 잘 구분하는지 시각적으로 확인
- Word Cloud 기반 XAI 해석 전, 모델의 전반적 구분 성능 이해

### 데이터 구성
- `target`: 실제 레이블
- `probs`: 모델의 클래스별 예측 확률 (softmax 결과)

### 결과 예시
- 각 클래스에 대해 ROC Curve가 그려지며, AUC(곡선 아래 면적)를 통해 성능 정량화
- AUC 값이 1에 가까울수록 해당 클래스에 대한 분류가 잘 되었음을 의미


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("validation_result.csv")
y_true = df["target"].values
y_score = np.vstack(df["preds"].values)

y_bin = label_binarize(y_true, classes=list(range(len(CLASS_NAMES))))
print(y_bin)
print(y_bin.shape)
n_classes = y_score.shape[1]

In [ ]:
plt.figure(figsize=(12, 8))
colors = plt.cm.tab20(np.linspace(0, 1, n_classes))

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_bin[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=colors[i], lw=2, label=f"{CLASS_NAMES[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('클래스별 ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()


## ROC Curve 해석 (Benign 클래스 기준)

- **True Positive Rate (TPR, 민감도)** vs **False Positive Rate (FPR)** 를 비교한 그래프입니다.
- 그래프 위쪽에 위치할수록 좋은 모델이며, **완벽한 분류기는 왼쪽 위 모서리에 가까운 곡선을 보입니다.**

---

### 현재 ROC Curve의 특징
- AUC (Area Under Curve) 값은 **0.72**로, **기준선(0.5)보다 높지만 우수한 성능은 아님**을 의미합니다.
- **Benign 클래스에 대한 구분 성능이 다소 떨어짐**을 시사합니다.
  - FPR이 높은 구간에서도 TPR이 완만하게 증가 → 혼동이 있다는 뜻.

---

### 왜 AUC는 낮을까?

> 전체 정확도나 F1-score는 높지만, AUC는 낮을 수 있는 대표적인 이유:

1. **클래스 불균형**  
   - Benign 클래스가 데이터셋에서 매우 큰 비중을 차지할 경우, ROC는 오히려 성능을 낮게 평가할 수 있습니다.  
   - 즉, 다수 클래스의 'True Negative'가 많아 **FPR이 높아지기 쉬움**.

2. **모델의 출력 확률 분포가 치우쳐 있을 경우**  
   - Benign으로 예측할 때 **확신(Confidence)이 높지 않거나**,  
     **다른 클래스와의 경계가 애매**할 경우, TPR이 낮아질 수 있습니다.

3. **ROC Curve는 '확률 예측의 품질'을 평가**  
   - 단순히 정답/오답이 아니라 **확률 예측이 얼마나 잘 구분되는지를 본다는 점**에서,  
     F1-score와 다른 성격의 지표입니다.

---

### 보충 설명

- 정확도, F1-score → **이진 결정 후의 결과 평가**
- ROC-AUC → **모델이 얼마나 잘 "점수 기반으로" 구분하는지를 평가**

---

### 결론

- **AUC가 낮다고 모델이 무조건 나쁘다는 뜻은 아님**
- 특히 **다중 클래스 분류 + 클래스 불균형 상황**에서는  
  **정확도나 F1-score와 AUC가 다르게 나올 수 있음**
- 다양한 지표를 함께 보고 판단해야 하며,  
  **Benign 클래스의 AUC 개선을 위해 정규화, 샘플링, threshold 조정 등을 고려**할 수 있음.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSV 파일 로딩
df = pd.read_csv("validation_result_all.csv")

df['class_name'] = CLASS_NAMES

# 그래프 스타일 설정
plt.figure(figsize=(12, 6))
plt.bar(df['class_name'], df['f1'], color='skyblue')
plt.xticks(rotation=45)
plt.title("클래스별 F1-score")
plt.xlabel("Class")
plt.ylabel("F1-score")
plt.ylim(0.7, 1.0)
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
display(df)

## 클래스별 성능 요약 테이블 해석

이 표는 모델이 각 클래스를 얼마나 잘 분류했는지를 보여주는 상세한 성능 분석 자료입니다.

| 항목 | 설명 |
|------|------|
| class | 클래스 ID (0: Benign, 1~11: 악성 유형별) |
| support | 해당 클래스의 실제 샘플 수 |
| precision | 해당 클래스로 예측한 것 중 실제 맞춘 비율 |
| recall | 실제 해당 클래스를 모델이 얼마나 잘 맞췄는지 |
| f1 | precision과 recall의 조화 평균 |
| accuracy_classwise | TP + TN / 전체, 해당 클래스 중심 정확도 |
| tp / fp / fn / tn | 혼동 행렬 기반 지표 |
| prevalence | 전체 샘플 중 해당 클래스 비중 (불균형 확인용) |

---

### 주요 포인트 요약

- **Benign (class 0)** 은 데이터의 34.7%를 차지하며, recall=0.9992로 거의 완벽하게 감지됨
- **Banking 관련 (class 1~3)** 은 전체의 51.5%를 차지하고 있으며 precision/recall 모두 높음
- **저빈도 클래스 (e.g., class 8 Ransomware, class 7 Malspam)** 은 recall이 0.8 수준으로 상대적으로 낮음 → 모델이 일부 놓치고 있음
- **Recon_C2 (class 9)** 는 데이터 비중은 적지만 상대적으로 높은 f1 점수 확보
- 전반적으로 precision과 recall의 균형이 양호하며, class imbalance에도 불구하고 모델이 잘 학습된 모습

---


## ☁️ WordCloud 기반 XAI (설명용)

- 각 클래스의 텍스트 데이터를 수집하고, 등장 빈도가 높은 단어들을 시각화하여
  모델이 어떤 단어에 반응했을 가능성이 높은지를 파악합니다.
  
- WordCloud는 다음과 같은 목적에 활용할 수 있습니다:
  - 클래스별 주요 키워드 직관적 비교
  - 오탐/미탐 사례에서 공통된 단어 또는 특징 추출
  - 초보자에게 XAI 개념을 쉽게 설명하는 도구로 적합

---

### 예시

- **Benign** 클래스:
  - 정상 사이트 관련 단어: `user-agent`, `host`, `keep-alive`, `accept`
- **Malspam/Phishing** 클래스:
  - 악성 이메일, 다운로드 관련 단어: `exe`, `zip`, `form`, `invoice`, `click`

---

In [ ]:
from utils import data_load
# data setting
data_base_dir = "./model_data/"
bert_file = "bert_inputs.txt"
MAXLEN = 512
nlp_data_label = data_load(base_dir=data_base_dir, bert_flie=bert_file, MAXLEN=MAXLEN)

#### Data setting
- payload만 활용하기 위해 전처리 진행
- ex)
 > 'Detected Protocol: HTTP(S) Entropy: 5.28 Encryption Status: Possibly Structured Data Signature: 474554202f6d6564 (first 8 bytes) Length: 3840 bytes Readable ASCII Data: GET /media/system/js/jquery-1.6.5.min.js HTTP/1.1Accept: */*Referer: http://[redacted] Accept-Language: en-USUser-Agent: Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0)Accept-Encoding: gzip, deflateHost: www.insightcrime.orgConnecti'
 
 >GET /media/system/js/jquery-1.6.5.min.js HTTP/1.1Accept: */*Referer: http://[redacted] Accept-Language: en-USUser-Agent: Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0)Accept-Encoding: gzip, deflateHost: www.insightcrime.orgConnecti


In [ ]:
labels_data = {}
nlp_datas = nlp_data_label[0]
nlp_labels = nlp_data_label[1]
for text, label in zip(nlp_datas, nlp_labels):
    temp_list = []
    if label in labels_data:
        temp_list = labels_data[label]
    else:
        temp_list = []
    temp_list.append(text.split("Readable ASCII Data: ")[-1])
    labels_data[label] = temp_list


In [ ]:
%pip install wordcloud

In [ ]:
from wordcloud import WordCloud
from collections import Counter

def get_wc(all_tokens, class_id, output_name=None):
    word_freq = Counter(all_tokens)
    wc = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
    # 시각화 및 저장
    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Class {class_id}: {CLASS_NAMES[class_id]}")
    plt.tight_layout()
    if output_name is None:
        plt.savefig(f"wordcloud_by_class/class_{class_id}_{CLASS_NAMES[class_id]}.png")
    else:
        plt.savefig(f"wordcloud_by_class/{output_name}.png")
    plt.close()

import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def is_valid_token(token):
    return (
        re.fullmatch(r"[a-z]{2,}", token)
        and token not in ENGLISH_STOP_WORDS
    )

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from utils.TH_BERT import tokenizer


os.makedirs("wordcloud_by_class", exist_ok=True)

for class_id in labels_data.keys():
    texts = labels_data.get(class_id)
    
    all_tokens = []
    for text in texts:
        tokens = tokenizer.tokenize(text.lower())
        tokens = [
            t for t in tokens 
            if t not in tokenizer.all_special_tokens
            and not t.startswith("##")
            and is_valid_token(t)
        ]
        all_tokens.extend(tokens)
    
    get_wc(all_tokens, class_id)

print("디토큰화 기반 WordCloud 저장 완료 (wordcloud_by_class 폴더)")

### Class 0: Benign - WordCloud 해석
![Benign WordCloud](./wordcloud_by_class/class_0_Benign.png)
- **주요 키워드:** `search`, `https`, `kr`, `root`, `ns`, `ss`, `version`, `agent`, `application`, `get`, `content`, `host`, `api`, `options`, `type`, `science`, `discover`, `length`, `site`, `data`

---

#### 해석 요약:

- 이 WordCloud는 모델이 **정상 트래픽(Benign)** 으로 분류한 데이터들에서 등장한 단어들을 시각화한 것입니다.
- 상위 키워드 대부분이 **정적 웹 페이지 구성 요소** 또는 **일반 API 호출 구조**에 가까운 단어들로 구성되어 있습니다.

---

#### 키워드 기반 분석:

| 키워드 | 의미 | 해석 |
|--------|------|------|
| `https`, `get`, `host`, `api`, `options`, `agent` | HTTP 통신에서 일반적으로 사용되는 요청 헤더 키워드 | 정상적인 웹브라우저 요청의 흔적 |
| `kr`, `co`, `com`, `org` | 도메인 또는 국가코드 | 실제 상업/연구 사이트 접속 가능성 |
| `science`, `search`, `discover` | 검색 및 정보 관련 키워드 | 검색 엔진 또는 학술 정보 페이지 등 |
| `application`, `version`, `content`, `type` | 소프트웨어/웹서버 메타 정보 | 브라우저 설정, 파일 MIME 등 일반 통신 요소 |

---

#### 전처리 설명

- 원래는 `/`, `%`, `@` 등의 **특수문자**가 포함되어 있었지만, WordCloud의 **가독성 향상**을 위해 제거함.
- 대신 의미 있는 단어들이 더 잘 드러나도록 **알파벳과 숫자 기반 키워드 중심으로 시각화**함.

---

#### 초심자를 위한 해석 팁

> "왜 이런 단어들이 중요한가요?"

- 악성 트래픽 탐지 모델을 만들 때, 어떤 요청이 정상인지 아닌지를 구분하는 기준은 **패턴의 차이**입니다.
- 정상 트래픽에서는 보통 `search`, `api`, `https`, `content` 등 **웹서비스 이용 과정에서 흔히 나타나는 단어**들이 포함됩니다.
- WordCloud는 이처럼 **빈도 중심의 시각화**를 통해 **정상과 비정상의 차이**를 직관적으로 보여주는 데 유용합니다.

---

#### SHAP, LIME은 왜 안썼나요?

- SHAP, LIME 등은 강력한 **모델 해석 도구**이지만, 초심자 입장에서 코드를 이해하고 활용하기에는 진입장벽이 높습니다.
- 이 실습에서는 먼저 WordCloud 기반의 **기초적인 인사이트 도출**을 통해 모델의 동작을 직관적으로 이해하는 것을 목표로 합니다.
- 이후 고급 해석 기법으로 확장할 수 있도록 학습 기반을 마련합니다.

---


# 클래스별 WordCloud 해석 (선택 클래스 3종)

복잡한 SHAP, LIME 등의 정량적 XAI 대신 **WordCloud**를 활용하여 각 클래스의 특징 단어를 시각적으로 해석했습니다. 특수문자 제거, 토큰화 및 디토큰화 전처리를 적용하였고, 그럼에도 불구하고 해석에 애매함이 존재할 수 있음을 감안해 설명을 진행합니다.

---

## 선정 클래스 요약

| Class ID | Class Name           | 선정 이유 요약                                                             |
|----------|----------------------|----------------------------------------------------------------------------|
| 1        | Banking_Emotet_Family| 메일/서버 관련 단어가 많아 공격 패턴 설명에 적합                           |
| 5        | RAT                  | `validation`, `control`, `online` 등 특징 단어가 명확하여 설명 가능        |
| 7        | Malspam_Phishing     | 피싱 공격에 연관된 단어 (`express`, `installer`, `origin` 등)가 다수 등장 |

---

## Class 1: `Banking_Emotet_Family`
![Benign WordCloud](./wordcloud_by_class/class_1_Banking_Emotet_Family.png)
- **주요 단어**: `https`, `com`, `mail`, `query`, `agent`, `server`, `cache`, `protocol`
- **해석**:
  - `mail`, `query`, `agent` → 메일을 기반으로 한 공격 흐름 가능성 시사
  - `cache`, `server`, `keep-alive` → 웹 기반 자원 요청 또는 중간자 공격 추정
  - `protocol`, `type`, `host` → HTTP 계열의 정형화된 구조 내 악성 요소 삽입 가능성

---

## Class 5: `RAT (Remote Access Trojan)`
![Benign WordCloud](./wordcloud_by_class/class_5_RAT.png)

- **주요 단어**: `com`, `validation`, `control`, `online`, `ua`, `hosting`, `fresh`, `trust`
- **해석**:
  - `control`, `validation`, `online` → 명령제어(C2) 및 클라이언트 상태 점검 과정 존재 시사
  - `hosting`, `fresh`, `trust` → 정상 호스팅/신뢰 단어로 위장한 악성 통신 특징
  - `ua`, `pt`, `rat`, `ov` 등 일부 코드/지표들은 분석상 오탐 가능성도 내포

---

## Class 7: `Malspam_Phishing`
![Benign WordCloud](./wordcloud_by_class/class_7_Malspam_Phishing.png)

- **주요 단어**: `http`, `keep`, `alive`, `origin`, `express`, `installer`, `found`, `allow`
- **해석**:
  - `express`, `origin`, `installer` → 설치 유도형 피싱 메일 혹은 웹 설치파일 다운로드 유도 가능성
  - `keep-alive`, `control`, `cache` → 지속 연결 유지 및 세션 악용 시도
  - `allow`, `not`, `we`, `us` → 사용자 권한 요청 및 허용 유도 시도 표현 포함

---

> ❗ 참고: WordCloud 기반 해석은 **정확한 공격 분석**이 아닌 **시각적 개연성 확인용**으로 사용됩니다. 실제 공격 분석에는 정량 분석 및 샘플 디코딩이 필수입니다.


In [ ]:
inaccuracy_df = pd.read_csv("inaccuracy_samples.csv")
wordcloud_tokens_by_class = {}  # 클래스별 토큰 저장용 dict
for class_id in sorted(inaccuracy_df["target"].unique()):
    texts = inaccuracy_df[inaccuracy_df['target'] == class_id]['text']
    if texts is not np.nan:
        print(texts)
    else:
        continue
    all_tokens = []
    for text in texts:
        try:
        	tokens = tokenizer.tokenize(text.lower())
        except AttributeError:
            continue
        tokens = [
            t for t in tokens 
            if t not in tokenizer.all_special_tokens
            and not t.startswith("##")
            and is_valid_token(t)
        ]
        all_tokens.extend(tokens)
    output_name = f"inaccuracy_wordcloud_{class_id}_{CLASS_NAMES[class_id]}"
    wordcloud_tokens_by_class[class_id] = all_tokens
    get_wc(all_tokens, class_id, output_name)


### 오탐 클래스 분석 - Class 0: Benign

![Benign WordCloud](./wordcloud_by_class/inaccuracy_wordcloud_0_Benign.png)

#### 주요 키워드
- **data**, **byte**, **protocol**, **det**, **status**, **length**, **entropy**, **content**, **signature**, **structure** 등

#### 분석 요약
Benign 클래스의 오탐 탐지에서 눈에 띄는 키워드들은 실제 사용자 데이터라기보다는 **프로토콜 분석**에서 발생한 용어들이 많이 포함되어 있습니다.

예를 들어:
- `entropy`, `signature`, `status`, `structure` 등의 단어는 **보안 도구의 분석 과정**에서 자동 생성된 메타 정보일 가능성이 높습니다.
- `possibly`, `read`, `det`, `protocol` 등의 단어들도 **추론 메시지 또는 분석 결과의 일부 표현**일 가능성이 있습니다.

이러한 특성 때문에, 모델은 이를 오히려 **악성 행위 탐지에 활용되는 정황 정보로 착각**할 수 있으며, **Benign임에도 불구하고 악성으로 잘못 분류하는 오탐**이 발생한 것으로 해석됩니다.

### Class 1: Banking_Emotet_Family
![Benign WordCloud](./wordcloud_by_class/inaccuracy_wordcloud_1_Banking_Emotet_Family.png)

#### 주요 단어
- protocol, query, answer, data, byte, read, flag, signature, status

#### 해석
Emotet은 전형적인 뱅킹 악성코드로, 초기 스팸 이메일 → 매크로 → 다운로더 → 모듈 설치의 흐름을 탑니다.
`query`, `answer`, `flag` 등은 내부에서 DNS/HTTP 요청을 통한 C2 연결을 시도하거나, 네트워크 기반 활동에서 파생된 것으로 추정됩니다.
`signature`, `status`, `possibly` 등은 탐지 로직이 적용된 흔적이기도 하며, 이로 인해 benign 트래픽과 구분이 어려워 오탐이 발생했을 수 있습니다.

### Class 3: Banking_Other
![Benign WordCloud](./wordcloud_by_class/inaccuracy_wordcloud_3_Banking_Other.png)

#### 주요 단어
- data, byte, http, user, windows, answer, agent, status

#### 해석
기타 뱅킹 악성코드를 포함한 이 클래스는 악성 행위가 비교적 모호하거나 다양한 유형이 혼합된 경우입니다.
특히 `http`, `windows`, `agent`, `user` 등은 정상적인 소프트웨어나 benign한 행위와 중복될 수 있어,
트래픽 구조가 일반적인 요청과 유사한 경우 오탐 가능성이 높습니다.
`agent`, `alive`, `host`는 HTTP Header 또는 프로토콜 레벨에서 공통적으로 등장하는 필드이며,
이 역시 탐지 모델이 혼동했을 여지가 있습니다.


### Class 9: Recon_C2
![Benign WordCloud](./wordcloud_by_class/inaccuracy_wordcloud_9_Recon_C2.png)
#### 주요 단어
- protocol, possibly, search, structure, det, read, signature, custom, status

#### 해석
Reconnaissance(정찰) 및 C2(명령제어) 행위가 섞인 클래스입니다.
`search`, `possibly`, `custom`, `protocol`, `signature` 등은 탐지된 트래픽이 명확한 악성 행위보다는 **의심스러운 요청** 또는 **비표준 포맷**을 가진 경우일 수 있습니다.
`custom`, `structure`, `entropy` 등의 단어는 비정형 트래픽(예: 특수 제작된 C2 패킷)을 나타낼 수 있고,
모델이 공격의 명확한 패턴을 잡기 어려운 상황에서 benign으로 판단해 오탐된 것으로 추정됩니다.



In [ ]:
def get_top_tokens(tokens, top_n=10):
    counter = Counter(tokens)
    return counter.most_common(top_n)

# 클래스별 토큰 저장된 dict 사용: wordcloud_tokens_by_class[class_id] = [token1, token2, ...]
selected_classes = {
    0: "Benign",
    1: "Banking_Emotet_Family",
    3: "Banking_Other",
    9: "Recon_C2"
}

top10_by_class = {}

for class_id, label in selected_classes.items():
    tokens = wordcloud_tokens_by_class[class_id]  # ← 각 클래스별 토큰 리스트
    top10_by_class[label] = get_top_tokens(tokens, top_n=10)

# 출력
import pandas as pd

df_top10 = pd.DataFrame.from_dict(top10_by_class, orient="index").T
df_top10.columns.name = "Class"
display(df_top10)


## Word Frequency Top-10 by Class (디코딩 기반 WordCloud 분석)

각 클래스에서 추출된 디코딩 텍스트를 기반으로 토큰화한 결과, 등장 빈도가 높은 상위 10개 단어를 추출하였습니다.  
이 분석은 SHAP이나 LIME과 같은 복잡한 모델 해석 기법 대신, 초심자도 이해하기 쉬운 방식으로  
"모델이 어떤 단어를 중심으로 예측했는가"를 간접적으로 해석하기 위한 용도입니다.

---

### Class 0: Benign
| 순위 | 단어     | 빈도수 |
|------|----------|--------|
| 1    | `byte`   | 274    |
| 2    | `data`   | 242    |
| 3    | `det`    | 214    |
| 4    | `protocol` | 213  |
| 5    | `en`     | 183    |
| 6    | `length` | 168    |
| 7    | `read`   | 129    |
| 8    | `http`   | 120    |
| 9    | `entro`  | 111    |
| 10   | `status` | 111    |

- **해석**: benign 데이터에서는 일반적인 네트워크 구조 정보(`byte`, `protocol`, `length`, `status`)가 주로 등장하며,  
  민감하거나 명시적인 악성 동작을 의미하는 특이 패턴은 적음. 단어 `http`도 정상 통신의 흔한 구조로 볼 수 있음.

---

### Class 1: Banking_Emotet_Family
| 순위 | 단어       | 빈도수 |
|------|------------|--------|
| 1    | `protocol` | 3288   |
| 2    | `det`      | 3278   |
| 3    | `data`     | 3246   |
| 4    | `byte`     | 2804   |
| 5    | `read`     | 2321   |
| 6    | `en`       | 2014   |
| 7    | `quer`     | 1925   |
| 8    | `length`   | 1839   |
| 9    | `id`       | 1538   |
| 10   | `answer`   | 1459   |

- **해석**: `quer`, `id`, `answer` 등의 요청/응답 구조는 C2 서버 또는 피싱 시나리오에서 발생하는 행동일 가능성 있음.  
  `det`, `protocol`, `data` 등도 Emotet 악성 모듈이 활용하는 네트워크 통신 구조를 나타낼 수 있음.

---

### Class 2: Banking_Other
| 순위 | 단어       | 빈도수 |
|------|------------|--------|
| 1    | `data`     | 16818  |
| 2    | `byte`     | 11831  |
| 3    | `protocol` | 11295  |
| 4    | `det`      | 11289  |
| 5    | `http`     | 9830   |
| 6    | `read`     | 9368   |
| 7    | `en`       | 8299   |
| 8    | `length`   | 7023   |
| 9    | `possibly` | 5867   |
| 10   | `status`   | 5860   |

- **해석**: 일반적인 Banking 관련 악성코드에서도 `http`, `read`, `possibly`와 같이  
  의심스러운 다운로드, 정보 수집, 의도된 리다이렉션과 관련된 단어들이 자주 나타남.

---

### Class 9: Recon_C2
| 순위 | 단어       | 빈도수 |
|------|------------|--------|
| 1    | `data`     | 6601   |
| 2    | `protocol` | 5427   |
| 3    | `byte`     | 4612   |
| 4    | `possibly` | 4229   |
| 5    | `en`       | 4084   |
| 6    | `det`      | 3921   |
| 7    | `read`     | 3671   |
| 8    | `length`   | 2748   |
| 9    | `entro`    | 2504   |
| 10   | `status`   | 2504   |

- **해석**: Recon_C2 클래스에서는 `possibly`, `det`, `entro` 등의 불확실성/탐지 지표가 반복적으로 등장하며,  
  이는 탐색 및 명령제어(C2) 단계에서의 정보 수집 및 은폐 시도 가능성을 시사함.

---

이러한 분석은 **정확한 단어의 의미보다는 상대적인 등장 빈도와 조합**을 통해  
"어떤 클래스가 어떤 단어 패턴에 민감한가"를 이해하는 데 목적이 있습니다.
